In [1]:
# 🎨 RUN THIS FIRST!
from IPython.display import display, HTML; from pathlib import Path
display(HTML(f'<style>{Path("workshops/deploy-redis-for-developers-amr/module-01-redis-fundamentals/notebook-styles.css").read_text()}</style>'))

# Module 1: Redis Fundamentals - Interactive Lab

**Duration:** 60 minutes  
**Format:** Hands-On Interactive Lab  
**Level:** Foundation

---

## 🎯 Lab Overview

In this interactive lab, you'll:
- �� Set up a local Redis instance using Docker
- 🔧 Connect to Redis with Python (redis-py)
- 📊 Work hands-on with all 5 core data structures
- 🎮 Build practical examples (counters, queues, leaderboards)
- 🧪 Test Redis commands with CLI-style output

**Prerequisites:**
- Docker running in Codespaces (pre-configured)
- Basic Python knowledge
- Completed Module 1 README (theory)

---

## Part 1: Setup Docker Redis Container

### 🐳 Start Local Redis Instance

We'll run Redis in a Docker container for this lab. This gives us a clean, isolated environment.

In [4]:
# Stop and remove existing container (if any)
print("🧹 Cleaning up existing Redis containers...")
subprocess.run("docker stop redis-fundamentals 2>/dev/null", shell=True, capture_output=True)
subprocess.run("docker rm redis-fundamentals 2>/dev/null", shell=True, capture_output=True)
print("✅ Cleanup complete")
print()

# Pull Redis image (if not exists)
print("📦 Pulling Redis image...")
result = subprocess.run(
    "docker pull redis:latest",
    shell=True,
    capture_output=True,
    text=True
)
if "up to date" in result.stdout or "Downloaded" in result.stdout:
    print("✅ Redis image ready")
print()

# Start Redis container
print("🚀 Starting Redis container...")
result = subprocess.run(
    "docker run -d --name redis-fundamentals -p 6379:6379 redis:latest",
    shell=True,
    capture_output=True,
    text=True
)

if result.returncode == 0:
    container_id = result.stdout.strip()[:12]
    print(f"✅ Redis container started")
    print(f"   Container ID: {container_id}")
    print(f"   Port: 6379")
    print(f"   Host: localhost")
else:
    print(f"❌ Failed to start container: {result.stderr}")

# Wait for Redis to be ready
print("\n⏳ Waiting for Redis to be ready...")
time.sleep(2)
print("✅ Redis is ready!")

🧹 Cleaning up existing Redis containers...
✅ Cleanup complete

📦 Pulling Redis image...
✅ Redis image ready

🚀 Starting Redis container...
❌ Failed to start container: docker: Error response from daemon: failed to set up container networking: driver failed programming external connectivity on endpoint redis-fundamentals (7c20668d7c1d7009089740aed8f0c4b6c3eaf3a5386487105db65381bcec85f1): failed to bind host port for 0.0.0.0:6379:172.17.0.3:6379/tcp: address already in use

Run 'docker run --help' for more information


⏳ Waiting for Redis to be ready...
✅ Redis image ready

🚀 Starting Redis container...
❌ Failed to start container: docker: Error response from daemon: failed to set up container networking: driver failed programming external connectivity on endpoint redis-fundamentals (7c20668d7c1d7009089740aed8f0c4b6c3eaf3a5386487105db65381bcec85f1): failed to bind host port for 0.0.0.0:6379:172.17.0.3:6379/tcp: address already in use

Run 'docker run --help' for more information


⏳ Wai

### 📦 Install and Import Python Redis Client

In [5]:
# Install redis-py
!pip install -q redis
print("✅ redis-py installed")

✅ redis-py installed


In [6]:
import redis
import json
from IPython.display import display, HTML, Markdown

# Create Redis connection
r = redis.Redis(
    host='localhost',
    port=6379,
    decode_responses=True,  # Get strings instead of bytes
    socket_connect_timeout=5
)

# Test connection
print("🔗 Testing Redis connection...")
try:
    r.ping()
    print("✅ Connected to Redis successfully!")
    print(f"   Redis version: {r.info('server')['redis_version']}")
except Exception as e:
    print(f"❌ Connection failed: {e}")

🔗 Testing Redis connection...
✅ Connected to Redis successfully!
   Redis version: 7.0.15


### 🎨 Helper Function: CLI-Style Output

Let's create a helper to display Redis commands like the redis-cli does:

In [7]:
def redis_cli(command):
    """
    Execute Redis command using real redis-cli in Docker container.
    
    Args:
        command: Redis command as string (e.g., "SET key value")
    """
    # Execute command via docker exec
    full_cmd = f'docker exec redis-fundamentals redis-cli {command}'
    result = subprocess.run(
        full_cmd,
        shell=True,
        capture_output=True,
        text=True
    )
    
    # Format output with CLI-style display
    output = result.stdout.strip() if result.stdout else result.stderr.strip()
    
    # Display in CLI style
    display(HTML(f'''
    <div style="
        background: #1e1e1e;
        color: #d4d4d4;
        padding: 12px 16px;
        border-radius: 6px;
        font-family: 'Courier New', monospace;
        font-size: 14px;
        margin: 8px 0;
        border-left: 4px solid #3b82f6;
    ">
        <div style="color: #4ec9b0; margin-bottom: 4px;">
            <span style="color: #569cd6;">redis&gt;</span> {command}
        </div>
        <div style="color: #ce9178; padding-left: 60px;">
            {output if output else '(nil)'}
        </div>
    </div>
    '''))
    
    return output

print("✅ Helper function defined")
print("\n📝 Usage example:")
print('   redis_cli("SET key value")')
print('   redis_cli("GET key")')
print("\n💡 This uses real redis-cli inside the Docker container!")

✅ Helper functions defined

📝 Usage examples:
   execute_redis('SET', 'key', 'value')
   execute_redis('GET', 'key')
   redis_cli('EXAMPLE command', '(result)')


### 🧪 Test CLI-Style Output

In [8]:
# Test the CLI-style display with real redis-cli
print("🎬 Testing real redis-cli commands:\n")

redis_cli("PING")
redis_cli("SET test:key 'Hello Redis!'")
redis_cli("GET test:key")
redis_cli("DEL test:key")

print("\n✅ Real redis-cli working!")

🎬 Testing CLI-style output:




✅ CLI-style output working!


---

## Part 2: Strings - Counters and Simple Values

### 📝 Basic String Operations

In [1]:
print("🎯 Demo: Basic String Operations\n")

# SET and GET
redis_cli("SET user:1000:name 'John Doe'")
redis_cli("GET user:1000:name")

# SET with expiration (3600 seconds = 1 hour)
redis_cli("SETEX session:abc123 3600 user_session_data")

# Check TTL (time to live)
redis_cli("TTL session:abc123")

# SET only if not exists
redis_cli("SETNX lock:resource1 locked")
redis_cli("SETNX lock:resource1 locked")  # Will fail (already exists)

# Clean up
redis_cli("DEL lock:resource1")

🎯 Demo: Basic String Operations



NameError: name 'redis_cli' is not defined

### 🔢 Counters with INCR/DECR

In [ ]:
print("🎯 Demo: Page View Counter\n")

# Initialize counter
redis_cli("SET page:/home:views 0")

# Simulate page views
print("\n📊 Simulating 5 page views:\n")
for i in range(5):
    redis_cli("INCR page:/home:views")
    time.sleep(0.1)  # Small delay for visual effect

# Add bulk views
print("\n📊 Adding 100 more views in bulk:\n")
redis_cli("INCRBY page:/home:views 100")

# Get final count
print("\n📊 Final view count:\n")
redis_cli("GET page:/home:views")

### 📚 Multiple Operations with MSET/MGET

In [ ]:
print("🎯 Demo: Batch Operations\n")

# Set multiple keys at once
redis_cli("MSET user:1:name Alice user:2:name Bob user:3:name Charlie")

# Get multiple keys at once
print("\n📖 Retrieving all users:\n")
redis_cli("MGET user:1:name user:2:name user:3:name")

---

## Part 3: Lists - Queues and Feeds

### 📋 Basic List Operations

In [ ]:
print("🎯 Demo: Task Queue with Lists\n")

# Push tasks to the queue (right push = enqueue)
execute_redis('RPUSH', 'queue:tasks', 'task1: Process order #100')
execute_redis('RPUSH', 'queue:tasks', 'task2: Send email to user@example.com')
execute_redis('RPUSH', 'queue:tasks', 'task3: Generate report for Q4')

# Check queue length
print("\n📊 Queue status:\n")
execute_redis('LLEN', 'queue:tasks')

# View all tasks
print("\n📋 All tasks in queue:\n")
execute_redis('LRANGE', 'queue:tasks', 0, -1)

# Process tasks (left pop = dequeue)
print("\n⚙️  Processing tasks:\n")
execute_redis('LPOP', 'queue:tasks')
execute_redis('LPOP', 'queue:tasks')

# Check remaining tasks
print("\n📋 Remaining tasks:\n")
execute_redis('LRANGE', 'queue:tasks', 0, -1)

### 📰 Activity Feed Example

In [ ]:
print("🎯 Demo: User Activity Feed\n")

# Add activities (newest first with LPUSH)
execute_redis('LPUSH', 'user:123:feed', 'Followed user:456')
time.sleep(0.2)
execute_redis('LPUSH', 'user:123:feed', 'Liked post #789')
time.sleep(0.2)
execute_redis('LPUSH', 'user:123:feed', 'Posted a photo')
time.sleep(0.2)
execute_redis('LPUSH', 'user:123:feed', 'Commented on post #500')

# Get recent 3 activities
print("\n📰 Recent 3 activities:\n")
execute_redis('LRANGE', 'user:123:feed', 0, 2)

# Trim feed to keep only 100 most recent items
print("\n✂️  Trimming feed to 100 items:\n")
execute_redis('LTRIM', 'user:123:feed', 0, 99)

---

## Part 4: Sets - Unique Collections

### 🏷️ Basic Set Operations

In [ ]:
print("🎯 Demo: Blog Post Tags\n")

# Add tags to posts
execute_redis('SADD', 'tags:post:1', 'redis', 'database', 'nosql', 'caching')
execute_redis('SADD', 'tags:post:2', 'python', 'redis', 'tutorial')
execute_redis('SADD', 'tags:post:3', 'azure', 'cloud', 'redis')

# Check if tag exists
print("\n🔍 Checking if post:1 has 'redis' tag:\n")
execute_redis('SISMEMBER', 'tags:post:1', 'redis')

# Get all tags
print("\n🏷️  All tags for post:1:\n")
execute_redis('SMEMBERS', 'tags:post:1')

# Count tags
print("\n📊 Number of tags:\n")
execute_redis('SCARD', 'tags:post:1')

### 🔀 Set Operations: Union, Intersection, Difference

In [ ]:
print("🎯 Demo: Tag-Based Post Discovery\n")

# Create sets of post IDs for each tag
execute_redis('SADD', 'tag:redis:posts', 'post:1', 'post:2', 'post:3')
execute_redis('SADD', 'tag:python:posts', 'post:2', 'post:4', 'post:5')
execute_redis('SADD', 'tag:azure:posts', 'post:3', 'post:6', 'post:7')

# Find posts tagged with BOTH redis AND python (intersection)
print("\n🔍 Posts tagged with redis AND python:\n")
execute_redis('SINTER', 'tag:redis:posts', 'tag:python:posts')

# Find posts tagged with redis OR azure (union)
print("\n🔍 Posts tagged with redis OR azure:\n")
execute_redis('SUNION', 'tag:redis:posts', 'tag:azure:posts')

# Find posts tagged with azure but NOT redis (difference)
print("\n🔍 Posts tagged with azure but NOT redis:\n")
execute_redis('SDIFF', 'tag:azure:posts', 'tag:redis:posts')

### �� Unique Visitor Tracking

In [ ]:
print("🎯 Demo: Unique Daily Visitors\n")

# Simulate visitors (duplicates are automatically ignored)
visitors = ['user:100', 'user:200', 'user:100', 'user:300', 'user:200', 'user:400']

print("📊 Adding visitors (some duplicates):\n")
for visitor in visitors:
    execute_redis('SADD', 'visitors:2025-11-19', visitor)
    time.sleep(0.1)

# Count unique visitors
print("\n📊 Total unique visitors:\n")
execute_redis('SCARD', 'visitors:2025-11-19')

# Get all unique visitors
print("\n👥 All unique visitors:\n")
execute_redis('SMEMBERS', 'visitors:2025-11-19')

---

## Part 5: Hashes - Object Storage

### 👤 User Profile with Hashes

In [ ]:
print("🎯 Demo: User Profile Management\n")

# Create user profile
execute_redis('HMSET', 'user:5000',
              'username', 'alice',
              'email', 'alice@example.com',
              'created_at', '2025-01-15',
              'last_login', '2025-11-19',
              'login_count', '42')

# Get full profile
print("\n👤 Full user profile:\n")
execute_redis('HGETALL', 'user:5000')

# Get specific fields
print("\n📧 Get username and email:\n")
execute_redis('HMGET', 'user:5000', 'username', 'email')

# Update single field
print("\n🔄 Updating last login:\n")
execute_redis('HSET', 'user:5000', 'last_login', '2025-11-20T10:30:00Z')

# Increment login count
print("\n📊 Incrementing login count:\n")
execute_redis('HINCRBY', 'user:5000', 'login_count', 1)

### 🛍️ Product Catalog Example

In [ ]:
print("🎯 Demo: Product Catalog\n")

# Create product
execute_redis('HMSET', 'product:1001',
              'name', 'Redis in Action Book',
              'price', '39.99',
              'stock', '150',
              'category', 'books',
              'rating', '4.8')

# Get product info
print("\n🛍️  Product details:\n")
execute_redis('HGETALL', 'product:1001')

# Check stock
print("\n📦 Current stock:\n")
execute_redis('HGET', 'product:1001', 'stock')

# Decrease stock (sale)
print("\n💰 Processing sale (decreasing stock):\n")
execute_redis('HINCRBY', 'product:1001', 'stock', -1)

# Check if field exists
print("\n🔍 Does product have 'discount' field?:\n")
execute_redis('HEXISTS', 'product:1001', 'discount')

---

## Part 6: Sorted Sets - Rankings and Leaderboards

### 🏆 Game Leaderboard

In [ ]:
print("🎯 Demo: Real-Time Game Leaderboard\n")

# Add players with scores
execute_redis('ZADD', 'game:leaderboard',
              1500, 'alice',
              2200, 'bob',
              1800, 'charlie',
              2500, 'diana',
              1900, 'eve')

# Get top 3 players (highest scores)
print("\n🏆 Top 3 Players:\n")
execute_redis('ZREVRANGE', 'game:leaderboard', 0, 2, 'WITHSCORES')

# Get player rank (0-indexed, highest score = rank 0)
print("\n📊 Diana's rank:\n")
execute_redis('ZREVRANK', 'game:leaderboard', 'diana')

# Get player score
print("\n🎮 Charlie's score:\n")
execute_redis('ZSCORE', 'game:leaderboard', 'charlie')

# Player scores more points
print("\n⚡ Alice scores 500 points!\n")
execute_redis('ZINCRBY', 'game:leaderboard', 500, 'alice')

# Updated top 3
print("\n🏆 Updated Top 3:\n")
execute_redis('ZREVRANGE', 'game:leaderboard', 0, 2, 'WITHSCORES')

### 📈 Trending Posts by Engagement

In [ ]:
print("🎯 Demo: Trending Posts\n")

# Add posts with engagement scores
execute_redis('ZADD', 'trending:posts',
              45, 'post:101',
              120, 'post:102',
              89, 'post:103',
              200, 'post:104',
              67, 'post:105')

# Get top 3 trending
print("\n🔥 Top 3 Trending Posts:\n")
execute_redis('ZREVRANGE', 'trending:posts', 0, 2, 'WITHSCORES')

# Get posts in engagement range (50-100)
print("\n📊 Posts with 50-100 engagement:\n")
execute_redis('ZRANGEBYSCORE', 'trending:posts', 50, 100, 'WITHSCORES')

# Count posts above threshold
print("\n📈 Posts with 100+ engagement:\n")
execute_redis('ZCOUNT', 'trending:posts', 100, '+inf')

# Get leaderboard size
print("\n📊 Total posts in trending:\n")
execute_redis('ZCARD', 'trending:posts')

---

## Part 7: Practical Use Cases

### 🚦 Rate Limiting Implementation

In [ ]:
print("🎯 Demo: Simple Rate Limiter (Fixed Window)\n")

def check_rate_limit(user_id, limit=5):
    """Simple rate limiter: 5 requests per minute"""
    key = f"rate_limit:{user_id}:{int(time.time() // 60)}"
    
    # Increment counter
    current = r.incr(key)
    
    # Set expiry on first request
    if current == 1:
        r.expire(key, 60)
    
    # Check limit
    allowed = current <= limit
    
    return allowed, current

# Simulate API requests
user = 'user:123'
print(f"📊 Rate limit: 5 requests per minute for {user}\n")

for i in range(7):
    allowed, count = check_rate_limit(user)
    status = "✅ ALLOWED" if allowed else "❌ BLOCKED"
    print(f"Request {i+1}: {status} (count: {count}/5)")
    time.sleep(0.1)

### 💾 Caching Pattern (Cache-Aside)

In [ ]:
print("🎯 Demo: Cache-Aside Pattern\n")

# Simulate database
fake_database = {
    'user:1000': {'id': 1000, 'name': 'Alice', 'email': 'alice@example.com'},
    'user:2000': {'id': 2000, 'name': 'Bob', 'email': 'bob@example.com'}
}

def get_user_with_cache(user_id):
    """Get user with cache-aside pattern"""
    cache_key = f"cache:user:{user_id}"
    
    # Try cache first
    cached = r.get(cache_key)
    if cached:
        print(f"✅ CACHE HIT for user:{user_id}")
        return json.loads(cached)
    
    # Cache miss - fetch from database
    print(f"⚠️  CACHE MISS for user:{user_id} - fetching from database...")
    user_data = fake_database.get(f'user:{user_id}')
    
    if user_data:
        # Store in cache with TTL
        r.setex(cache_key, 60, json.dumps(user_data))
        print(f"💾 Stored in cache (TTL: 60s)")
    
    return user_data

# First access - cache miss
print("\n🔍 First access to user:1000:\n")
user1 = get_user_with_cache(1000)
print(f"   Data: {user1}\n")

# Second access - cache hit
print("🔍 Second access to user:1000:\n")
user2 = get_user_with_cache(1000)
print(f"   Data: {user2}\n")

# Check cache TTL
print("⏱️  Cache TTL:\n")
execute_redis('TTL', 'cache:user:1000')

### 🔔 Pub/Sub Messaging Demo

In [ ]:
print("🎯 Demo: Publish/Subscribe Messaging\n")

# Publish messages to a channel
channel = 'notifications:user:1000'

print(f"📢 Publishing messages to channel: {channel}\n")

messages = [
    'New follower: user:2000',
    'Someone liked your post',
    'New comment on your photo'
]

for msg in messages:
    subscribers = execute_redis('PUBLISH', channel, msg)
    print(f"   Message: '{msg}'")
    print(f"   Delivered to {subscribers} subscribers\n")
    time.sleep(0.2)

print("\n💡 Note: Pub/Sub is fire-and-forget. Messages are only delivered to active subscribers.")
print("   For guaranteed delivery, use Redis Streams (covered in Module 9).")

---

## Part 8: Redis Server Information

### 📊 Server Stats and Configuration

In [ ]:
print("🎯 Redis Server Information\n")

# Server info
info = r.info('server')
print("📦 Server:")
print(f"   Redis Version: {info['redis_version']}")
print(f"   OS: {info['os']}")
print(f"   Architecture: {info['arch_bits']}-bit")
print(f"   Uptime: {info['uptime_in_seconds']} seconds\n")

# Memory info
info = r.info('memory')
used_mb = info['used_memory'] / 1024 / 1024
peak_mb = info['used_memory_peak'] / 1024 / 1024
print("💾 Memory:")
print(f"   Used: {used_mb:.2f} MB")
print(f"   Peak: {peak_mb:.2f} MB")
print(f"   Fragmentation Ratio: {info.get('mem_fragmentation_ratio', 'N/A')}\n")

# Stats
info = r.info('stats')
print("📊 Stats:")
print(f"   Total Connections: {info['total_connections_received']}")
print(f"   Total Commands: {info['total_commands_processed']}")
print(f"   Keyspace Hits: {info.get('keyspace_hits', 0)}")
print(f"   Keyspace Misses: {info.get('keyspace_misses', 0)}")

# Calculate hit rate
hits = info.get('keyspace_hits', 0)
misses = info.get('keyspace_misses', 0)
if hits + misses > 0:
    hit_rate = (hits / (hits + misses)) * 100
    print(f"   Cache Hit Rate: {hit_rate:.1f}%\n")

# Database size
dbsize = r.dbsize()
print(f"🗄️  Database:")
print(f"   Total Keys: {dbsize}")

### 🧹 Database Cleanup

In [ ]:
print("🎯 Database Cleanup\n")

# Get current key count
before = r.dbsize()
print(f"📊 Keys before cleanup: {before}")

# Flush current database
print("\n🧹 Flushing database...")
r.flushdb()

# Verify
after = r.dbsize()
print(f"✅ Keys after cleanup: {after}")
print(f"\n💡 Removed {before - after} keys")

---

## Part 9: Cleanup Docker Container

### 🛑 Stop and Remove Redis Container

In [ ]:
print("🛑 Stopping Redis container...\n")

# Stop container
result = subprocess.run(
    "docker stop redis-fundamentals",
    shell=True,
    capture_output=True,
    text=True
)

if result.returncode == 0:
    print("✅ Container stopped")
else:
    print(f"⚠️  {result.stderr}")

# Remove container
result = subprocess.run(
    "docker rm redis-fundamentals",
    shell=True,
    capture_output=True,
    text=True
)

if result.returncode == 0:
    print("✅ Container removed")
else:
    print(f"⚠️  {result.stderr}")

print("\n🎉 Cleanup complete!")
print("\n💡 To restart Redis for future labs, run the cells in Part 1 again.")

---

## 🎯 Lab Summary

### ✅ What You Accomplished

1. **Environment Setup**
   - ✅ Started Redis in Docker container
   - ✅ Connected with redis-py
   - ✅ Created CLI-style command display

2. **Hands-On with Data Structures**
   - ✅ **Strings**: Counters, caching, simple values
   - ✅ **Lists**: Queues, activity feeds
   - ✅ **Sets**: Unique collections, tag filtering
   - ✅ **Hashes**: User profiles, product catalogs
   - ✅ **Sorted Sets**: Leaderboards, trending items

3. **Practical Patterns**
   - ✅ Rate limiting
   - ✅ Cache-aside pattern
   - ✅ Pub/Sub messaging
   - ✅ Real-time analytics

4. **Monitoring & Management**
   - ✅ Server information
   - ✅ Memory usage
   - ✅ Performance stats
   - ✅ Database cleanup

---

### 🎓 Key Skills Acquired

- 🐳 **Docker**: Run Redis in containers
- �� **Python**: Use redis-py client
- �� **Data Structures**: Choose appropriate types for use cases
- 🏗️ **Patterns**: Implement common Redis patterns
- 🔧 **Operations**: Monitor and manage Redis instances

---

### 🚀 Next Steps

Continue your Redis journey:

**Module 2: Azure Managed Redis Architecture**
- Enterprise Redis deployment
- SKU selection and sizing
- High availability patterns
- Production configurations

**Module 7: Provision & Connect Lab**
- Deploy Azure Managed Redis
- Entra ID authentication
- Secure connections
- Production-ready setup

**Module 8: Implement Caching Lab**
- Advanced caching strategies
- Cache invalidation
- Performance optimization
- Real-world scenarios

---

### 📚 Additional Resources

**Official Documentation:**
- [Redis Commands Reference](https://redis.io/commands/)
- [Redis Data Types Tutorial](https://redis.io/docs/data-types/)
- [redis-py Documentation](https://redis-py.readthedocs.io/)

**Interactive Learning:**
- [Try Redis](https://try.redis.io/) - Browser-based Redis
- [Redis University](https://university.redis.com/) - Free courses

**Tools:**
- [RedisInsight](https://redis.com/redis-enterprise/redis-insight/) - GUI client
- [redis-cli](https://redis.io/docs/ui/cli/) - Command-line interface

---

## 🎉 Congratulations!

You've completed the Redis Fundamentals Interactive Lab!

You now have hands-on experience with:
- ✅ All 5 core Redis data structures
- ✅ Common use cases and patterns
- ✅ Python redis-py client
- ✅ Docker containerization
- ✅ Real-world examples

**Ready for the next module?** Continue to Module 2 for Azure Managed Redis!

---